---
toc: false
page-layout: full
---

# Week 6B<br>Web Scraping

- Section 401
- Wednesday, October 11, 2023

## Week 6 agenda: web scraping

**Last time:**
- Why web scraping? 
- Getting familiar with the Web
- Web scraping: extracting data from static sites

**Today:**
- Practice with web scraping
- How to deal with dynamic content

In [1]:
# Start with the usual imports
# We'll use these throughout
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Part 1: Web scraping exercises

For each of the exercises, use the Web Inspector to inspect the structure of the relevant web page, and identify the HTML content you will need to scrape with Python.


### 1. The number of votes for Cherelle Parker in the May mayoral primary

The relevant URL is: [https://philadelphiaresults.azurewebsites.us/ResultsSW.aspx?type=MAY&map=CTY](https://philadelphiaresults.azurewebsites.us/ResultsSW.aspx?type=MAY&map=CTY)

**Hint:** We're interested in just a single HTML element so you can inspect the website, identify the right element, and copy the selector for the element.

### 2. How many millions of people are currently experiencing drought?

Relevant URL: [https://www.drought.gov/current-conditions](https://www.drought.gov/current-conditions)

**Hint:** We're interested in just a single HTML element so you can inspect the website, identify the right element, and copy the selector for the element.

### 3. Scrape the Weitzman School directory

The Weitzman School lists their directory of people on this page: [https://www.design.upenn.edu/people/list](https://www.design.upenn.edu/people/list). From this site, let's extract out following information:

- The person's name;
- title, and;
- associated department.

::: {.callout-tip}
This example is similar to the Inquirer Clean Plates demo from last lecture. 

The info we want for each person is wrapped up in a `<div>` element. You can select all of those elements, loop over each one in a "for" loop, extract the three pieces of content we want from each `<div>`, and then save the result to a list.
:::


## Part 2: What about dynamic content?

How do you scrape data that is loaded via Javascript or only appears after user interaction?

<center>
    <img src="imgs/yoda.png" width=700>
</center>

::: {.callout-note}
### Note: web browser needed

You'll need a web browser installed to use `selenium`, e.g., FireFox, Google Chrome, Edge, etc.
:::

### Selenium 

- Designed as a framework for testing webpages during development
- Provides an interface to interact with webpages just as a user would 
- Becoming increasingly popular for web scraping dynamic content from pages

### Two common use cases

Two cases when `requests.get()` won't work:

1. Many sites load content via javascript, so requests.get() won't return anything. It doesn't load any javascript on the page, just returns the static HTML content.
1. If the site requires user interaction via buttons, dropdowns, etc., then get requests won't be able to show that information.


**But how to solve them?**

1. Look on the "Network" tab for the external requests that the javascript is using to pull the data. 
1. Selenium to the rescue! 

Let's try it out!

### Example #1: Philly's Property App

- URL: [https://property.phila.gov/](https://property.phila.gov/)
- Let's see if we can extract the property assessment values for my old apartment in Fairmount

**Problem:** We'll need to enter an address, click a button, and THEN scrape the information.

#### 1. Initialize the web driver

The initialization steps will depend on which browser you want to use!

In [229]:
# Import the webdriver from selenium
from selenium import webdriver

::: {.callout-important}
#### Important: Working on Binder

If you are working on Binder, you'll need to use FireFox in "headless" mode, which prevents a browser window from opening.

If you are working locally, it's better to run with the default options — you'll be able to see the browser window open and change as we perform the web scraping.
:::

##### Using Google Chrome

In [249]:
# UNCOMMENT BELOW TO USE CHROME

driver = webdriver.Chrome()

##### Using Firefox

If you are working on Binder, use the below code!

In [250]:
# UNCOMMENT BELOW IF ON BINDER

# options = webdriver.FirefoxOptions()

# IF ON BINDER, RUN IN "HEADLESS" MODE (NO BROWSER WINDOW IS OPENED)
# COMMENT THIS LINE IF WORKING LOCALLY
# options.add_argument("--headless")

# Initialize
#driver = webdriver.Firefox(options=options)

##### Using Microsoft Edge

In [251]:
# UNCOMMENT BELOW TO USE MICROSOFT EDGE

#driver = webdriver.Edge()

#### 2. Navigate to the URL

In [252]:
url = "https://property.phila.gov/"
driver.get(url)

#### 3. Select the address input element

After it loads, let's take a look in the web inspector. We'll need to identify the search bar so we can input our desired address.

It looks like the search bar has a class of `"pvm-search-control-input"`.

In [253]:
# Use the dot syntax for class selectors
address_input_selector = ".pvm-search-control-input"

We can select it directly in selenium using the `driver.find_element()` function and telling selenium we are using a css selector to do the selection.

In [254]:
from selenium.webdriver.common.by import By

In [256]:
# Select the address input by the element's CSS selector
address_input = driver.find_element(By.CSS_SELECTOR, address_input_selector)

#### 4. Type the address

Use the `send_keys()` function to enter the text into the input element:

In [263]:
# Input our example address
address_input.send_keys("1739 Wallace St #102")

#### 5. Select the search button and click it

Using the web inspector, it looks like both the clear and search buttons have the class of `"pvm-search-control-button"`. Let's select both, choose the second button, and click it.

In [267]:
button_selector = ".pvm-search-control-button"

# Get the button element
clear_button, search_button = driver.find_elements(By.CSS_SELECTOR, button_selector)

In [268]:
# Click it
search_button.click()

#### 6. Use BeautifulSoup to parse the results

- Use the `page_source` attribute to get the current HTML displayed on the page
- Initialize a "soup" object with the HTML

In [269]:
propertySoup = BeautifulSoup(driver.page_source, "html.parser")

From the web inspector, it looks like the table we want has an ID of `"ownerProperties"`. Let's select all of the elements with that ID:

In [270]:
# Use the # for ID selector
table_selector = "#ownerProperties"

tables = propertySoup.select(table_selector)

In [271]:
len(tables)

3

::: {.callout-caution}

### Watch out! 

We selected by element ID, which *should* be unique but sometimes isn't! Here we see that three tables all share the same ID on the page. 

:::

The table we want is the second table on the page with the ID "ownerProperties" — let's select it.

In [274]:
tables[1]

<table class="stack" data-v-49670532="" id="ownerProperties" role="grid"><thead data-v-49670532=""><tr data-v-49670532=""><th class="" data-v-49670532="" title=""> Year </th><th class="" data-v-49670532="" title=""> Market Value </th><th class="" data-v-49670532="" title=""> Taxable Land </th><th class="" data-v-49670532="" title=""> Taxable Improvement </th><th class="" data-v-49670532="" title=""> Exempt Land </th><th class="" data-v-49670532="" title=""> Exempt Improvement </th></tr></thead><tbody data-v-49670532=""><tr class="" data-v-49670532="" data-v-6ed23da8=""><td class="" data-v-6ed23da8="" sorttable_customkey="2024"><div data-v-2ca357c0="" data-v-6ed23da8=""><div data-v-2ca357c0=""></div><!-- --></div><b data-v-6ed23da8="" style="display: none;"><!-- --><div data-v-6ed23da8="">2024</div></b><div data-v-6ed23da8=""><!-- --><div data-v-6ed23da8=""><div data-v-6ed23da8="">2024</div><!-- --></div></div></td><td class="" data-v-6ed23da8="" sorttable_customkey="291400"><div data-v

In [275]:
table = tables[1]

Select all the rows in the table (the `<tr>` elements):

In [276]:
# Select all row (<tr>) elements in the table
rows = table.select("tr")

Now, we can take a look at each `<tr>` element and extract each cell's data in the `<td>` cell

In [281]:
print(tr.prettify())

<tr class="" data-v-49670532="" data-v-6ed23da8="">
 <td class="" data-v-6ed23da8="" sorttable_customkey="2015">
  <div data-v-2ca357c0="" data-v-6ed23da8="">
   <div data-v-2ca357c0="">
   </div>
   <!-- -->
  </div>
  <b data-v-6ed23da8="" style="display: none;">
   <!-- -->
   <div data-v-6ed23da8="">
    2015
   </div>
  </b>
  <div data-v-6ed23da8="">
   <!-- -->
   <div data-v-6ed23da8="">
    <div data-v-6ed23da8="">
     2015
    </div>
    <!-- -->
   </div>
  </div>
 </td>
 <td class="" data-v-6ed23da8="" sorttable_customkey="209400">
  <div data-v-2ca357c0="" data-v-6ed23da8="">
   <div data-v-2ca357c0="">
   </div>
   <!-- -->
  </div>
  <b data-v-6ed23da8="" style="display: none;">
   <!-- -->
   <div data-v-6ed23da8="">
    $209,400
   </div>
  </b>
  <div data-v-6ed23da8="">
   <!-- -->
   <div data-v-6ed23da8="">
    <div data-v-6ed23da8="">
     $209,400
    </div>
    <!-- -->
   </div>
  </div>
 </td>
 <td class="" data-v-6ed23da8="" sorttable_customkey="20940">
  <d

It looks like the content for each cell is inside a `<b>` tag inside a `<td>` cell:

In [283]:
# Empty list to store scraped data
scraped_data = []

# Loop over all rows but skip header row
for row in rows[1:]:
    #
    # Create a list for each cell in this particular row
    row_data = [cell.text for cell in row.select("td b")]

    # Save it
    scraped_data.append(row_data)

scraped_data

[['2024', '$291,400', '$43,700', '$247,700', '$0', '$0'],
 ['2023', '$291,400', '$43,700', '$247,700', '$0', '$0'],
 ['2022', '$264,900', '$39,735', '$225,165', '$0', '$0'],
 ['2021', '$264,900', '$39,735', '$225,165', '$0', '$0'],
 ['2020', '$264,900', '$39,735', '$225,165', '$0', '$0'],
 ['2019', '$264,900', '$39,735', '$185,165', '$0', '$40,000'],
 ['2018', '$240,800', '$38,528', '$172,272', '$0', '$30,000'],
 ['2017', '$240,800', '$38,528', '$172,272', '$0', '$30,000'],
 ['2016', '$209,400', '$20,940', '$158,460', '$0', '$30,000'],
 ['2015', '$209,400', '$20,940', '$158,460', '$0', '$30,000']]

Make it into a DataFrame:

In [284]:
assessments = pd.DataFrame(
    scraped_data,
    columns=[
        "Year",
        "Market Value",
        "Taxable Land",
        "Taxable Improvement",
        "Exempt Land",
        "Exempt Improvement",
    ],
)

assessments

,Year,Market Value,Taxable Land,Taxable Improvement,Exempt Land,Exempt Improvement
0,2024,"$291,400","$43,700","$247,700",$0,$0
1,2023,"$291,400","$43,700","$247,700",$0,$0
2,2022,"$264,900","$39,735","$225,165",$0,$0
3,2021,"$264,900","$39,735","$225,165",$0,$0
4,2020,"$264,900","$39,735","$225,165",$0,$0
5,2019,"$264,900","$39,735","$185,165",$0,"$40,000"
6,2018,"$240,800","$38,528","$172,272",$0,"$30,000"
7,2017,"$240,800","$38,528","$172,272",$0,"$30,000"
8,2016,"$209,400","$20,940","$158,460",$0,"$30,000"
9,2015,"$209,400","$20,940","$158,460",$0,"$30,000"


Success!

### Example #2: Scraping the Philadelphia Municipal Courts portal

- URL: [https://ujsportal.pacourts.us/CaseSearch](https://ujsportal.pacourts.us/CaseSearch)
- Given a Police incident number, we'll see if there is an associated court case with the incident

**Problem:** we'll need to click several buttons before we can see the info we want!

### Run the scraping analysis

Strategy:

- Rely on the Web Inspector to identify specific elements of the webpage
- Use Selenium to interact with the webpage
    - Change dropdown elements
    - Click buttons

#### 1. Open the URL

In [285]:
# Open the URL
url = "https://ujsportal.pacourts.us/CaseSearch"
driver.get(url)

#### 2. Create a dropdown "Select" element

We'll need to: 
- Select the dropdown element on the main page by its ID
- Initialize a `selenium` `Select()` object

In [286]:
# Use the Web Inspector to get the css selector of the dropdown select element
dropdown_selector = "#SearchBy-Control > select"

In [287]:
# Select the dropdown by the element's CSS selector
dropdown = driver.find_element(By.CSS_SELECTOR, dropdown_selector)

Create the Select object:

In [288]:
from selenium.webdriver.support.ui import Select

# Initialize a Select object
dropdown_select = Select(dropdown)

#### 3. Change the selected text in the dropdown

Change the selected element: "Police Incident/Complaint Number" 

In [289]:
# Set the selected text in the dropdown element
dropdown_select.select_by_visible_text("Incident Number")

#### 4. Set the incident number 

In [290]:
# Get the input element for the DC number
incident_input_selector = "#IncidentNumber-Control > input"
incident_input = driver.find_element(By.CSS_SELECTOR, incident_input_selector)

In [291]:
# Clear any existing entry
incident_input.clear()

# Input our example incident number
incident_input.send_keys("1725088232")

#### 5. Click the search button!

In [292]:
# Submit the search
search_button_selector = "#btnSearch"
driver.find_element(By.CSS_SELECTOR, search_button_selector).click()

#### 6. Use BeautifulSoup to parse the results

- Use the `page_source` attribute to get the current HTML displayed on the page
- Initialize a "soup" object with the HTML

In [293]:
courtsSoup = BeautifulSoup(driver.page_source, "html.parser")

Now we can:
- Identify the element holding all of the results
- Within this container, find the `<table>` element and each `<tr>` element within the table

In [294]:
# Select the results container by its ID 
results_table = courtsSoup.select_one("#caseSearchResultGrid")

In [295]:
# Get all of the <tr> rows inside the tbody element 
# NOTE: we using nested selections here!
results_rows = results_table.select("tbody > tr")

**Example:** The number of court cases

In [296]:
# Number of court cases
number_of_cases = len(results_rows)
print(f"Number of courts cases: {number_of_cases}")

Number of courts cases: 2


**Example:** Extract the text elements from the first row of the results

In [297]:
first_row = results_rows[0]

In [298]:
print(first_row.prettify())

<tr class="slide-active">
 <td class="display-none">
  1
 </td>
 <td class="display-none">
  0
 </td>
 <td>
  MC-51-CR-0030672-2017
 </td>
 <td>
  Common Pleas
 </td>
 <td>
  Comm. v. Velquez, Victor
 </td>
 <td>
  Closed
 </td>
 <td>
  10/13/2017
 </td>
 <td>
  Velquez, Victor
 </td>
 <td>
  09/05/1974
 </td>
 <td>
  Philadelphia
 </td>
 <td>
  MC-01-51-Crim
 </td>
 <td>
  U0981035
 </td>
 <td>
  1725088232-0030672
 </td>
 <td>
  1725088232
 </td>
 <td class="display-none">
 </td>
 <td class="display-none">
 </td>
 <td class="display-none">
 </td>
 <td class="display-none">
 </td>
 <td>
  <div class="grid inline-block">
   <div>
    <div class="inline-block">
     <a class="icon-wrapper" href="/Report/CpDocketSheet?docketNumber=MC-51-CR-0030672-2017&amp;dnh=%2FGgePQykMpAymRENgxLBzg%3D%3D" target="_blank">
      <img alt="Docket Sheet" class="icon-size" src="https://ujsportal.pacourts.us/resource/Images/svg-defs.svg?v=qJ77ypOpzSMFk7r1gsI6H0xjdteha_ZIjvGslGgQV2M#icon-document-letter-D" 

In [299]:
# Extract out all of the "<td>" cells from the first row
td_cells = first_row.select("td")

# Loop over each <td> cell
for cell in td_cells:
    
    # Extract out the text from the <td> element
    text = cell.text
    
    # Print out text
    if text != "":
        print(text)

1
0
MC-51-CR-0030672-2017
Common Pleas
Comm. v. Velquez, Victor
Closed
10/13/2017
Velquez, Victor
09/05/1974
Philadelphia
MC-01-51-Crim
U0981035
1725088232-0030672
1725088232
Docket SheetCourt Summary


#### 7. Close the driver!

In [300]:
driver.close()

## That's it!

- Next week: Part 2 of "getting data" with APIs
- See you on Monday!